# Лабораторная работа 1. Введение в машинное обучение. Обучение с учителем. Задача регрессии

### Преобразование датасета

In [2]:
import pandas as pd

data = pd.read_csv("../data/regression/trip_duration_task.csv").drop_duplicates()
data

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
0,id1080784,2.0,2016-02-29 16:40:21,2016-02-29 16:47:01,1,-73.953918,40.778873,-73.963875,40.771164,400
1,id0889885,1.0,2016-03-11 23:35:37,2016-03-11 23:53:57,2,-73.988312,40.731743,-73.994751,40.694931,1100
2,id0857912,2.0,2016-02-21 17:59:33,2016-02-21 18:26:48,2,-73.997314,40.721458,-73.948029,40.774918,1635
3,id3744273,2.0,2016-01-05 09:44:31,2016-01-05 10:03:32,6,-73.961670,40.759720,-73.956779,40.780628,1141
4,id0232939,1.0,2016-02-17 06:42:23,2016-02-17 06:56:31,1,-74.017120,40.708469,-73.988182,40.740631,848
...,...,...,...,...,...,...,...,...,...,...
729317,id3905982,2.0,2016-05-21 13:29:38,2016-05-21 13:34:34,2,-73.965919,40.789780,-73.952637,40.789181,296
729318,id0102861,1.0,2016-02-22 00:43:11,2016-02-22 00:48:26,1,-73.996666,40.737434,-74.001320,40.731911,315
729319,id0439699,1.0,2016-04-15 18:56:48,2016-04-15 19:08:01,1,-73.997849,40.761696,-74.001488,40.741207,673
729320,id2078912,1.0,2016-06-19 09:50:47,2016-06-19 09:58:14,1,-74.006706,40.708244,-74.013550,40.713814,447


In [ ]:
from matplotlib import pyplot as plt

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15, 10))
for idx, feature in enumerate(data.columns[:-1]):
    data.plot(feature, "trip_duration", subplots=True, kind="scatter", ax=axes[idx // 3, idx % 3])

In [2]:
data.drop(["id"], axis=1)
data = data.head(1500)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1500 entries, 0 to 1499
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 1500 non-null   object 
 1   vendor_id          1492 non-null   float64
 2   pickup_datetime    1500 non-null   object 
 3   dropoff_datetime   1500 non-null   object 
 4   passenger_count    1500 non-null   int64  
 5   pickup_longitude   1500 non-null   float64
 6   pickup_latitude    1497 non-null   float64
 7   dropoff_longitude  1500 non-null   float64
 8   dropoff_latitude   1500 non-null   float64
 9   trip_duration      1500 non-null   int64  
dtypes: float64(5), int64(2), object(3)
memory usage: 128.9+ KB


### Заполнение пустых строк

In [3]:
for column in list(data):
  if data[column].dtype == 'float64':
    valuesNotNAN = data[column].dropna()
    data[column] = data[column].fillna(round(sum(valuesNotNAN)/len(valuesNotNAN)))

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1500 entries, 0 to 1499
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 1500 non-null   object 
 1   vendor_id          1500 non-null   float64
 2   pickup_datetime    1500 non-null   object 
 3   dropoff_datetime   1500 non-null   object 
 4   passenger_count    1500 non-null   int64  
 5   pickup_longitude   1500 non-null   float64
 6   pickup_latitude    1500 non-null   float64
 7   dropoff_longitude  1500 non-null   float64
 8   dropoff_latitude   1500 non-null   float64
 9   trip_duration      1500 non-null   int64  
dtypes: float64(5), int64(2), object(3)
memory usage: 128.9+ KB


C:\Users\schir\AppData\Local\Temp\ipykernel_8044\3047176938.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column] = data[column].fillna(round(sum(valuesNotNAN)/len(valuesNotNAN)))


### Перевод категориальных признаков в числовой тип данных

In [4]:
import category_encoders as ce

columns = [column for column in list(data) if data[column].dtype == 'object']
encoder=ce.OneHotEncoder(cols=columns,handle_unknown='return_nan',return_df=True,use_cat_names=True)

data_encoded = encoder.fit_transform(data)
data_encoded.to_csv("../data/regression/trip_duration_update.csv")
display(data_encoded)

,id_id1080784,id_id0889885,id_id0857912,id_id3744273,id_id0232939,id_id1918069,id_id2429028,id_id1663798,id_id2436943,id_id2933909,...,dropoff_datetime_2016-04-15 15:21:20,dropoff_datetime_2016-02-18 11:38:43,dropoff_datetime_2016-05-13 08:14:27,dropoff_datetime_2016-01-30 10:16:02,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,-73.953918,40.778873,-73.963875,40.771164,400
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2,-73.988312,40.731743,-73.994751,40.694931,1100
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2,-73.997314,40.721458,-73.948029,40.774918,1635
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6,-73.961670,40.759720,-73.956779,40.780628,1141
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,-74.017120,40.708469,-73.988182,40.740631,848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2,-73.873161,40.774174,-73.992004,40.726036,2327
1496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1,-73.953667,40.778629,-73.951881,40.768620,398
1497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1,-73.964066,40.765232,-73.959404,40.771381,347
1498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1,-73.953529,40.767311,-73.980881,40.746082,1153


### Выделение целевого признака и предикторов

In [5]:
df = pd.read_csv("../data/regression/trip_duration_update.csv")

y = df["trip_duration"]
X = df.drop(['trip_duration'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Columns: 4508 entries, Unnamed: 0 to trip_duration
dtypes: float64(4505), int64(3)
memory usage: 51.6 MB


In [6]:
y

0        400
1       1100
2       1635
3       1141
4        848
        ... 
1495    2327
1496     398
1497     347
1498    1153
1499     290
Name: trip_duration, Length: 1500, dtype: int64

In [7]:
X

,Unnamed: 0,id_id1080784,id_id0889885,id_id0857912,id_id3744273,id_id0232939,id_id1918069,id_id2429028,id_id1663798,id_id2436943,...,dropoff_datetime_2016-06-04 17:50:23,dropoff_datetime_2016-04-15 15:21:20,dropoff_datetime_2016-02-18 11:38:43,dropoff_datetime_2016-05-13 08:14:27,dropoff_datetime_2016-01-30 10:16:02,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,-73.953918,40.778873,-73.963875,40.771164
1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2,-73.988312,40.731743,-73.994751,40.694931
2,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2,-73.997314,40.721458,-73.948029,40.774918
3,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6,-73.961670,40.759720,-73.956779,40.780628
4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1,-74.017120,40.708469,-73.988182,40.740631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,1495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,2,-73.873161,40.774174,-73.992004,40.726036
1496,1496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1,-73.953667,40.778629,-73.951881,40.768620
1497,1497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1,-73.964066,40.765232,-73.959404,40.771381
1498,1498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1,-73.953529,40.767311,-73.980881,40.746082


### Разделение данных на обучающую и тестовую выборки

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1200, 4507), (1200,), (300, 4507), (300,))

## Линейная регрессия

In [10]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge

lr = LinearRegression().fit(X_train, y_train)

lr.predict(X_test)

array([ 921.15316842,  845.28467958,  642.91794385, 1291.93831176,
        798.60156122,  635.20870807, 1473.07377894,  695.94852656,
       1183.27465024, 1247.00066126,  749.71406362, 1210.33507825,
       1109.71504728, 1004.93877626,  835.32499465,  764.66317388,
        574.23032422,  662.93984903,  847.83158612, 1169.22315577,
        690.16487234,  879.25699322,  747.7073707 ,  887.38368899,
        707.17762281, 1143.41645744,  794.05161243, 1079.06450411,
        914.47939497,  991.71657556,  868.53321163, 1021.73379476,
       1206.99979522,  940.6220522 ,  892.56602304, 1044.82042665,
       1475.31678099,  946.12193178,  809.93628867, 1373.37792417,
        954.50790929, 1858.04259949, 1836.10204254,  970.77359159,
       1041.21256457,  859.77488746, 1167.56663732,  714.80720935,
       1178.20634864, 1040.62659544,  979.14376287, 1068.66052134,
        582.98123053,  717.277585  ,  713.95681919,  580.98012928,
       1075.77347243, 1051.73784309,  547.85193791,  890.48699

# Метрики качества

In [11]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from math import sqrt

y_pred = lr.predict(X_test)

print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {lr.score(X_test, y_test)}')

MAE: 512.120958215506
MSE: 520554.72223803797
RMSE: 721.4947832368838
MAPE: 1.3200570132526668
R^2: 0.0737187061474519


In [12]:
len(lr.coef_)
lr.coef_

array([-1.69487690e-01, -2.52809168e+02,  3.49981139e+01, ...,
       -1.29243388e+03,  3.30095661e+03, -1.73052449e+03])

# Регуляризация в линейной регрессии (L1 Ridge)

In [13]:
# L1
ridge = Ridge(alpha=0.1).fit(X_train, y_train)
y_pred = ridge.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {ridge.score(X_test, y_test)}')
ridge.coef_

MAE: 512.5895987869719
MSE: 521600.6248373011
RMSE: 722.2192359922997
MAPE: 1.319177677000933
R^2: 0.07185761456283624


array([-1.69582615e-01, -2.44800579e+02,  3.42009000e+01, ...,
       -1.26663282e+03,  3.22285003e+03, -1.69107151e+03])

# Регуляризация в линейной регрессии (L2 Lasso)

In [17]:
# L2
lasso = Lasso(alpha=0.9).fit(X_train, y_train)
y_pred = lasso.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {lasso.score(X_test, y_test)}')
lasso.coef_

MAE: 440.71430912498073
MSE: 438065.8870414064
RMSE: 661.8654599247542
MAPE: 1.1999573252993212
R^2: 0.22050032531291186


array([ 1.56690149e-02, -0.00000000e+00,  0.00000000e+00, ...,
       -4.07245982e+02,  2.10333701e+03, -3.17550901e+03])

# Полиномиальная регрессия

In [15]:
from sklearn.preprocessing import PolynomialFeatures
pf = PolynomialFeatures(2)  

X_test_p=pf.fit_transform(X_test) 
X_train_p = pf.fit_transform(X_train)

MemoryError: Unable to allocate 22.7 GiB for an array with shape (300, 10163286) and data type float64

In [ ]:
lr2 = LinearRegression().fit(X_train, y_train)
lr2.coef_

# Метрики качества

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from math import sqrt

y_pred = lr.predict(X_test_p)

print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {lr.score(X_test_p, y_test)}')